# API Calls and Database Creation

### Attribution: [Data provided by Financial Modeling Prep](https://financialmodelingprep.com/developer/docs/), Yahoo! Finance

In [2]:
import yfinance as yf
from urllib.request import urlopen
import certifi
import json
from sqlalchemy import create_engine
import psycopg2
import numpy as np
import pandas as pd
import os
import configparser

### Tickers

In [2]:
# Retrieve Current S&P Stock Tickers - as of 08/22/2024
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_tickers = pd.read_html(url)[0]["Symbol"].to_list()

# Handling Errors from Wiki
sp500_tickers.remove("BRK.B")
sp500_tickers.remove("BF.B")
sp500_tickers.extend(["BRK-B", "BF-B"])

### Yfinance API

In [3]:
# Download and format the stock data without multi-index, "Date" column is currently stored as a datetime obj

# Download using yfinance API
df = yf.download(sp500_tickers, start="2023-01-01", end="2024-08-22", interval="1d")

# Formatting
df.sort_index(axis=1, inplace=True)
df = df.stack(future_stack=True).reset_index()  # future_stack required due to future deprecation
df.columns.name = None

# Visual Inspection - instead of multi-index, each stock shown for each day, future transformations require groupby(["Ticker").transform()
print(df.shape)
df.head()

[*********************100%%**********************]  503 of 503 completed


(206733, 8)


,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2023-01-03,A,148.440598,150.039993,153.130005,148.470001,151.960007,1414300.0
1,2023-01-03,AAL,12.740000,12.740000,13.000000,12.530000,12.910000,21865100.0
2,2023-01-03,AAPL,123.904633,125.070000,130.899994,124.169998,130.279999,112117500.0
3,2023-01-03,ABBV,151.773148,162.380005,163.020004,160.809998,162.039993,4937500.0
4,2023-01-03,ABNB,84.900002,84.900002,88.139999,84.300003,87.385002,3901200.0


### Financial Modeling Prep

In [5]:
# FMP requires an API key, using my environment to protect the specific key
api_key = os.getenv('STOCK_API_KEY')

In [22]:
from urllib.request import urlopen
import certifi
import json
import ssl


def get_jsonparsed_data(url):
    """
    The cafile=certifi.where() part ensures that urlopen uses the certificate bundle provided by certifi 
    to verify the SSL certificate of the URL you're trying to access. Then read the api response and return it in json-form

    Args:
    ------------
    url:str | url with api key to successfully download information

    Returns:
    ------------
    json.loads(data):json | json object with the results of the query

    Raises:
    ------------
    HTTPError | if any part of the query string is inaccurate, users must have their own API key "HTTP Error 401: Unauthorized"

    Example : 
    url = ("https://financialmodelingprep.com/api/v3/search?query=AA&apikey=YOUR_API_KEY")
    get_jsonparsed_data(url)
    """

    # Create a custom SSL context
    context = ssl.create_default_context(cafile=certifi.where())
    
    # Initialize response from FMP
    response = urlopen(url, context=context)  
    
    # Parse the response
    data = response.read().decode("utf-8")  # read the API response
    
    # Return as JSON object
    return json.loads(data)

# Demonstrate returned object for AAPL

## Options:
# "https://financialmodelingprep.com/api/v3/profile/AAPL?apikey=YOUR_API_KEY"  -> company profile!
# "https://financialmodelingprep.com/api/v3/search?query=AAPL&apikey={api_key}" -> symbol|name|currency|exchange ONLY

# Profile
url = (f"https://financialmodelingprep.com/api/v3/profile/AAPL?apikey={api_key}")
all_background = get_jsonparsed_data(url)
all_background

[{'symbol': 'AAPL',
  'price': 225.31,
  'beta': 1.244,
  'volAvg': 64811409,
  'mktCap': 3425635771000,
  'lastDiv': 1,
  'range': '164.08-237.23',
  'changes': 0.78,
  'companyName': 'Apple Inc.',
  'currency': 'USD',
  'cik': '0000320193',
  'isin': 'US0378331005',
  'cusip': '037833100',
  'exchange': 'NASDAQ Global Select',
  'exchangeShortName': 'NASDAQ',
  'industry': 'Consumer Electronics',
  'website': 'https://www.apple.com',
  'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital c

In [23]:
import json
import ssl
import certifi
import psycopg2
from urllib.request import urlopen


# Function to insert data into PostgreSQL table
def insert_data_to_postgres(connection, stock_profile):
    with connection.cursor() as cursor:
        insert_query = """
        INSERT INTO sp500_profiles (symbol, company_name, sector, industry, price, description, website)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            stock_profile.get("symbol"),
            stock_profile.get("companyName"),
            stock_profile.get("sector"),
            stock_profile.get("industry"),
            stock_profile.get("price"),
            stock_profile.get("description"),
            stock_profile.get("website"),
        ))
        connection.commit()

In [5]:
# Load environment variables from .env file
# load_dotenv()

db_name = os.getenv('stock_db')
# db_user = os.getenv('DB_USER')
db_password = os.getenv('db_password')
db_host = os.getenv('db_host')
# db_port = os.getenv('DB_PORT')

def create_sp500_table():
    with psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    ) as connection:
        with connection.cursor() as cursor:
            create_table_query = """
            CREATE TABLE IF NOT EXISTS sp500_profiles (
                id SERIAL PRIMARY KEY,
                symbol VARCHAR(10),
                company_name TEXT,
                sector TEXT,
                industry TEXT,
                price FLOAT,
                description TEXT,
                website TEXT
            );
            """
            cursor.execute(create_table_query)
            connection.commit()
            print("Table sp500_profiles created successfully.")

# create_sp500_table()


In [8]:
db_host, db_password, db_name

('localhost', 'Babypanda0726$', None)

In [ ]:
# List of S&P 500 stock symbols (use your list here)
sp500_tickers

# Initialize PostgreSQL connection
connection = psycopg2.connect(
    dbname="your_database",
    user="your_user",
    password="your_password",
    host="localhost",
    port="5432"
)

# Iterate over the list of S&P 500 stocks
api_key = "YOUR_API_KEY"  # Your FMP API key
for symbol in sp500_tickers:
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={api_key}"
    stock_data = get_jsonparsed_data(url)

    # Ensure the response is a list and contains data
    if stock_data and isinstance(stock_data, list):
        stock_profile = stock_data[0]  # The API returns a list with one profile
        insert_data_to_postgres(connection, stock_profile)

# Close the PostgreSQL connection
connection.close()